# Getting started with SCIFYsim

## Import the library

In [ ]:
import scifysim as sf
import numpy as np
import matplotlib.pyplot as plt
# optional (when using dark interfaces)
plt.style.use("dark_background")

In [ ]:
my_target = "GJ 86 A"

In [ ]:
print("SCIFYsim version: ", sf.version)
sf.logit.setLevel(sf.logging.ERROR)
t_exp =1.
seed = 10
expname = "R2000_base_sensitivity"
save_results = False

In [ ]:
pradii = np.linspace(0.1,3.,3)
#pradii = 1.0*np.ones(5)

In [ ]:
configfile = "config/chromatic_R2000.ini"

In [ ]:
asim = sf.utilities.prepare_all(configfile, thetarget=my_target, update_params=True,
                  instrumental_errors=True, seed=seed)
asim.combiner.chromatic_matrix(asim.lambda_science_range)

In [ ]:
diffuse = [asim.src.sky, asim.src.UT, asim.src.warm_optics, asim.src.combiner, asim.src.cold_optics]


# Making some exposure

## First, point the instrument to your target

In [ ]:
asim.point(asim.sequence[3], asim.target)

## Then you can start an integration

In [ ]:
integ = asim.make_metrologic_exposure(asim.src.planet, asim.src.star, diffuse,
                                      texp=t_exp)
integ.prepare_t_exp_base()
#consolidate_metrologic(integ)
integ.consolidate_metrologic()

## Nice view of the results

In [ ]:
shift_step = 1/(asim.n_spec_ch+2)
outputs = np.arange(integ.summed_signal.shape[2])
isources = np.arange(len(integ.sums))
raw_sources = [integ.static[0], integ.static[1],
               integ.static[2], integ.static[3],
               integ.static, integ.starlight, integ.planetlight]

In [ ]:
bottom = np.zeros_like(integ.sums[0])
pup = 1 # The pupil for which to plot the piston
print(integ.sums[0].shape)
signalplot = plt.figure(dpi=100)
bars = []
read_noise = integ.ron
for ksource, (thesource, label) in enumerate(zip(integ.sums, integ.source_labels)):
    photon_noise = np.sqrt(thesource)
    if ksource >= len(integ.static):
        inst_noise = np.std(raw_sources[ksource], axis=0)
    else:
        inst_noise = np.zeros((asim.lambda_science_range.shape[0], outputs.shape[0]))
    #print("Inst noise", ksource,  inst_noise.mean(axis=0))
    #print("Photon noise", ksource, photon_noise.mean(axis=0))
    noise = np.sqrt(photon_noise**2 + read_noise**2 + inst_noise**2)
    for ilamb in range(asim.lambda_science_range.shape[0]):
        #print(ksource, ilamb, label)
        #pdb.set_trace()
        if ilamb == 0:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                label=label, width=shift_step, color="C%d"%ksource)) #yerr=noise[ilamb,:]
        else:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                width=shift_step,  color="C%d"%ksource)) #yerr=noise[ilamb,:]
    bottom += thesource
#plt.legend((bars[i][0] for i in range(len(bars))), source_labels)
#Handled the legend with an condition in the loop
plt.legend(loc="upper left")
plt.xticks(outputs)
plt.xlabel(r"Output and spectral channel %.1f to %.1f $\mu m$ ($R\approx %.0f$)"%(asim.lambda_science_range[0]*1e6,
                                                                                 asim.lambda_science_range[-1]*1e6,
                                                                                 asim.R.mean()))
plt.title("Integration of %.2f s on %s"%(t_exp, asim.tarname))
plt.ylabel("Number of photons")
plt.show()

## Standard exposure


In [ ]:
dit = 1.5

In [ ]:
integ = asim.make_exposure(asim.src.planet, asim.src.star, diffuse,
                                texp=dit,
                                monitor_phase=False,
                               spectro=asim.spectro)

In [ ]:
from tqdm import tqdm
n_frames = 10
mynpix = 8
diffuse = [asim.src.sky, asim.src.UT, asim.src.warm_optics, asim.src.combiner, asim.src.cold_optics]
datacube = []
dit_intensity = []
for i in tqdm(range(n_frames)):
    integ = asim.make_exposure(asim.src.planet, asim.src.star, diffuse,
                                texp=dit,
                                monitor_phase=False,
                               spectro=asim.spectro)
    datacube.append(integ.get_total(spectrograph=None,
                                    t_exp=dit,
                                    n_pixsplit=mynpix))
    dit_intensity.append(integ.forensics["Expectancy"].sum(axis=0))
    integ.reset() # This can be removed after new kernel start
datacube = np.array(datacube)
dit_intensity = np.array(dit_intensity)

## The combiner matrix

In [ ]:

plt.style.use("default")

In [ ]:
from kernuller.diagrams import plot_chromatic_matrix
fig, axs, matrix = plot_chromatic_matrix(asim.combiner.M,
                                         sf.combiners.lamb, asim.lambda_science_range,
                                         verbose=False, returnmatrix=True,minfrac=0.9,
                                         plotout=True, show=False, title="With Tepper couplers")